# Power System Analysis 2025/26 @ IST Work assessment


In [75]:
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

First, we read the excel file with the network data and create a dictionary of datasets, one for each sheet. As we can see from the output of dfs.keys(), we now have five datasets: 'Nodes', 'Transformers', 'Generators', 'Lines' and 'Loads'.

In [76]:
path = "network_data.xlsx"
xls = pd.ExcelFile(path)

# Dictionary to hold DataFrames
dfs = {}

for sheet in xls.sheet_names:
    dfs[sheet] = pd.read_excel(xls, sheet_name=sheet)

# Display the keys to verify loading
dfs.keys()

dict_keys(['Nodes', 'Transformers', 'Generators', 'Lines', 'Loads'])

The next step is to properly define the base system, as well a the Fortescue matrix to use in later calculations. The base currents will be calculated based on the "Nodes" dataset, which presents the base voltages for each node of the system. Additionally, the base power is set to a fixed 100MVA.

Apart from this, the Resistances and Reactances were combined into a single Impedance collumn in all the datasets for easier handling - and converted from Ohm to per-unit when necessary.

In [77]:
# Define base system:

# Base system
Sb=100e6 # W (100 MVA)

# Compute base current
dfs["Nodes"]["Base Current [pu]"] = Sb/(math.sqrt(3)*dfs["Nodes"]["Base Voltage [V]"]) # A

# Transformation matrix
a = np.exp(2j * np.pi / 3)
A = np.array([[1, 1, 1],
              [1, a**2, a],
              [1, a, a**2]])

# Maximum fault parameters
tf_max = 2 # seconds
Zf_max_ohm = 40 # Ohm
Zf_max_pu = Zf_max_ohm/(dfs["Nodes"]["Base Voltage [V]"]**2/Sb)

# Compute the network impedances in pu

# Convert to the correct base 
dfs["Transformers"]["Z [pu]"] = (
    (dfs["Transformers"]["R [pu]"] + 1j * dfs["Transformers"]["X [pu]"])
    * (Sb / (dfs["Transformers"]["Power [MVA]"] * 1e6))
)

dfs["Generators"]["Z [pu]"] = (
    (dfs["Generators"]["R [pu]"] + 1j * dfs["Generators"]["X [pu]"])
    * (Sb / (dfs["Generators"]["Power [MVA]"] * 1e6))
)


dfs["Lines"]["Z1 [ohm]"] = dfs["Lines"]["R1 [ohm]"] + dfs["Lines"]["X1 [ohm]"]*1j
dfs["Lines"]["Z0 [ohm]"] = dfs["Lines"]["R0 [ohm]"] + dfs["Lines"]["X0 [ohm]"]*1j

dfs["Lines"]["Z1 [pu]"] = dfs["Lines"]["Z1 [ohm]"]/(dfs["Lines"]["Voltage Level [V]"]**2/Sb)
dfs["Lines"]["Z0 [pu]"] = dfs["Lines"]["Z0 [ohm]"]/(dfs["Lines"]["Voltage Level [V]"]**2/Sb)

dfs["Nodes"].head()

,Node ID Number,Base Voltage [V],Base Current [pu]
0,1,150000,384.900179
1,2,150000,384.900179
2,3,150000,384.900179
3,4,150000,384.900179
4,5,150000,384.900179


We can now build our primitive Impedance and Admittance matrices. Since all the transformers in this system have Ynd windings, none of them outright blocks zero sequence current to flow into the Y winding. As such, they should all be included in the primitive zero sequence matrices, since they allow zero sequence current to flow to ground, and as such, represent an impedance in the zero sequence network.

Aditionally, no loads were included since load current is usually negligible when performing short-circuit analysis at the grid level.

In [78]:
# Primitive Impedance matrices
# Created by placing the impedances of each element in the diagonal of a matrix, ordered lines - generators - transformers

Zprim1 = np.diag(
    dfs["Lines"]["Z1 [pu]"].tolist()
    + dfs["Generators"]["Z [pu]"].tolist()
    + dfs["Transformers"]["Z [pu]"].tolist()
)

Zprim0 = np.diag(
    dfs["Lines"]["Z0 [pu]"].tolist()
    + dfs["Generators"]["Z [pu]"].tolist()
    + dfs["Transformers"]["Z [pu]"].tolist()
)

# Primitive admittance matrices
# Created by inverting the primitive impedance matrices, since [Y] = [Z]^-1

Yprim1 = np.diag(1 / np.diag(Zprim1))
Yprim0 = np.diag(1 / np.diag(Zprim0))

In [79]:
# Check Lines
line_Z = dfs["Lines"]["Z1 [pu]"].tolist()
for i, z in enumerate(line_Z):
    assert Zprim1[i, i] == z, f"Mismatch in line {i}"

# Check Generators
gen_Z = dfs["Generators"]["Z [pu]"].tolist()
for i, z in enumerate(gen_Z):
    idx = len(line_Z) + i
    assert Zprim1[idx, idx] == z, f"Mismatch in generator {i}"

# Check Transformers
trafo_Z = dfs["Transformers"]["Z [pu]"].tolist()
for i, z in enumerate(trafo_Z):
    idx = len(line_Z) + len(gen_Z) + i
    assert Zprim1[idx, idx] == z, f"Mismatch in transformer {i}"

print("All Zprim1 entries match C1 row order")

All Zprim1 entries match C1 row order


Now we must create the constrain matrices for the zero and positive sequence. These matrices will have a dimmension of 12x20 (number of nodes x number of branches). The main difference between the two sequences will be that, for the zero sequence, the generator will not be connected to the transformers (since it is connected via a delta winding).

Aditionally, I have defined the currents to always flow from the lowest number node to the highest number node.

Since the positive sequence network has the exact same topology as the actual network, we can simply grab the node connections in our datasets to build the constrain matrix:

In [80]:
# Construct the positive sequence constrain matrix C1
C1 = np.zeros((20, 12), dtype=int)  # 20 branches, 12 nodes

# Lines
for branch_idx, row in dfs["Lines"].iterrows():
    node_a = int(row["Node A"]) - 1  # 0-based indexing
    node_b = int(row["Node B"]) - 1

    C1[branch_idx, node_a] = -1
    C1[branch_idx, node_b] =  1

# Generators
start_row = len(dfs["Lines"])  # where generator rows begin
for gen_idx, row in dfs["Generators"].iterrows():
    node = int(row["Connecting Nodes"]) - 1
    row_idx = start_row + gen_idx
    C1[row_idx, node] = 1

# Transformers
start_row += len(dfs["Generators"])
for trafo_idx, row in dfs["Transformers"].iterrows():
    node_h = int(row["Node H"]) - 1
    node_x = int(row["Node X"]) - 1
    row_idx = start_row + trafo_idx
    C1[row_idx, node_h] = -1
    C1[row_idx, node_x] =  1

print(C1)


[[-1  1  0  0  0  0  0  0  0  0  0  0]
 [-1  0  0  1  0  0  0  0  0  0  0  0]
 [-1  0  0  0  1  0  0  0  0  0  0  0]
 [ 0 -1  1  0  0  0  0  0  0  0  0  0]
 [ 0 -1  0  1  0  0  0  0  0  0  0  0]
 [ 0 -1  0  0  1  0  0  0  0  0  0  0]
 [ 0 -1  0  0  0  1  0  0  0  0  0  0]
 [ 0  0 -1  0  1  0  0  0  0  0  0  0]
 [ 0  0 -1  0  0  1  0  0  0  0  0  0]
 [ 0  0  0 -1  1  0  0  0  0  0  0  0]
 [ 0  0  0  0 -1  1  0  0  0  0  0  0]
 [ 0  0 -1  0  0  0  1  0  0  0  0  0]
 [ 0  0  0  0 -1  0  0  1  0  0  0  0]
 [ 0  0  0  0  0  0  0 -1  1  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  1  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  1  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  1]
 [-1  0  0  0  0  0  0  0  0  1  0  0]
 [ 0 -1  0  0  0  0  0  0  0  0  1  0]
 [ 0  0  0  0  0  0 -1  0  0  0  0  1]]


Now for the negative sequence, we have to keep in mind that the transformers are connected to neutral instead of the node on the delta side, which for this case is always the node on collumn "Node X". Aditionally, to keep the convention set that current always flows from low node to high node, and assuming the neutral is node 0, I considered that the "Node H" column now inputted a 1 instead of a -1. We can construct the constrain matrix the same way:

In [81]:
# Construct the positive sequence constrain matrix C0
C0 = np.zeros((20, 12), dtype=int)  # 20 branches, 12 nodes

# Lines
for branch_idx, row in dfs["Lines"].iterrows():
    node_a = int(row["Node A"]) - 1  # 0-based indexing
    node_b = int(row["Node B"]) - 1

    C0[branch_idx, node_a] = -1
    C0[branch_idx, node_b] =  1

# Generators
start_row = len(dfs["Lines"])  # where generator rows begin
for gen_idx, row in dfs["Generators"].iterrows():
    node = int(row["Connecting Nodes"]) - 1
    row_idx = start_row + gen_idx
    C0[row_idx, node] = 1

# Transformers
start_row += len(dfs["Generators"])
for trafo_idx, row in dfs["Transformers"].iterrows():
    node_h = int(row["Node H"]) - 1
    row_idx = start_row + trafo_idx
    C0[row_idx, node_h] = 1


print(C0)

[[-1  1  0  0  0  0  0  0  0  0  0  0]
 [-1  0  0  1  0  0  0  0  0  0  0  0]
 [-1  0  0  0  1  0  0  0  0  0  0  0]
 [ 0 -1  1  0  0  0  0  0  0  0  0  0]
 [ 0 -1  0  1  0  0  0  0  0  0  0  0]
 [ 0 -1  0  0  1  0  0  0  0  0  0  0]
 [ 0 -1  0  0  0  1  0  0  0  0  0  0]
 [ 0  0 -1  0  1  0  0  0  0  0  0  0]
 [ 0  0 -1  0  0  1  0  0  0  0  0  0]
 [ 0  0  0 -1  1  0  0  0  0  0  0  0]
 [ 0  0  0  0 -1  1  0  0  0  0  0  0]
 [ 0  0 -1  0  0  0  1  0  0  0  0  0]
 [ 0  0  0  0 -1  0  0  1  0  0  0  0]
 [ 0  0  0  0  0  0  0 -1  1  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  1  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  1  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  1]
 [ 1  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  1  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  1  0  0  0  0  0]]


We now have everything needed to compute the node impedance and admittance matrices, using the following equations:
$$
[\overline{Y}] = [K]^{t}[\overline{y}][K]
$$

$$
[\overline{Z}]^{-1} = [\overline{Y}]
$$

In [82]:
# Positive Sequence Node Admittance Matrix Computation
Y1 = C1.T @ Yprim1 @ C1

# Zero Sequence Node Admittance Matrix Computation
Y0 = C0.T @ Yprim0 @ C0

# Node Impedance Matrices
Z1 = np.linalg.inv(Y1)
Z0 = np.linalg.inv(Y0)


## Three Phase Fault

For a three phase fault, we have equation to calculate the fault current at each node:
$$
\overline{I}_{1-\Delta k} = \frac{\overline{V}_{1 - pF}}{\overline{Z}_{1-kk}}
$$

Which, since $\overline{V}_{1 - pF}$ is always 1 in per-unit, becomes:

$$
\overline{I}_{1-\Delta k} = \overline{Z}_{1-kk}^{-1}
$$

Additionally, it's important to state that for a three phase fault, the current in all 3 phases (a, b, c) is the same and equal to the positive sequence current.

In [83]:
Zkk_TP = np.diag(Z1)              # self-impedances
If_pu_TP = 1 / Zkk_TP              # fault currents in pu

Tp_TP = np.where(Zkk_TP.real != 0, Zkk_TP.imag / Zkk_TP.real, np.nan)

I_base = dfs["Nodes"]["Base Current [pu]"].values

I_seq = np.vstack([np.zeros_like(If_pu_TP), If_pu_TP, np.zeros_like(If_pu_TP)])

# Phase currents in pu
I_abc_pu = (A @ I_seq) / 3

# Convert to Amperes
I_abc_A = I_abc_pu * I_base

# Compute RMS and phase angle
I_abc_rms = np.abs(I_abc_A) / np.sqrt(2)

I_abc_angle = np.angle(I_abc_A, deg=True)

# Build DataFrame
n_nodes = I_abc_A.shape[1]
df_TP = pd.DataFrame({
    "Node": np.repeat(np.arange(n_nodes), 3),
    "TP Phase": np.tile(["A", "B", "C"], n_nodes),
    "TP Irms[A]": I_abc_rms.T.flatten(),
    "TP Phase[deg]": I_abc_angle.T.flatten(),
    "TP Tp[s]": np.repeat(Tp_TP, 3)
})

df_TP.head(9)

,Node,TP Phase,TP Irms[A],TP Phase[deg],TP Tp[s]
0,0,A,3929.624936,-88.557393,39.708443
1,0,B,3929.624936,151.442607,39.708443
2,0,C,3929.624936,31.442607,39.708443
3,1,A,2501.171023,-86.882959,18.363329
4,1,B,2501.171023,153.117041,18.363329
5,1,C,2501.171023,33.117041,18.363329
6,2,A,1270.374794,-84.475169,10.338432
7,2,B,1270.374794,155.524831,10.338432
8,2,C,1270.374794,35.524831,10.338432


## Single Line to Ground Fault

$$\bar{I}_{0-\Delta k} = \bar{I}_{1-\Delta k} = \bar{I}_{2-\Delta k} = \frac{\bar{V}_{1-pF}}{\bar{Z}_{0-kk} + \bar{Z}_{1-kk} + \bar{Z}_{2-kk} + 3R_f} = \frac{\bar{V}_{1-pF}}{\bar{Z}_T}$$

In [84]:
ZT_SLG = np.diag(Z1) + np.diag(Z1) + np.diag(Z0) + 3*Zf_max_pu
ZT_SLG = np.asarray(ZT_SLG)  # make sure it's a NumPy array

If_pu_SLG = 1.0 / ZT_SLG
Tp_SLG = np.where(ZT_SLG.real != 0, ZT_SLG.imag / ZT_SLG.real, np.nan)

# Fault current with zero fault impedance (bolted fault)
ZT_SLG_zero = np.diag(Z1) + np.diag(Z1) + np.diag(Z0)
ZT_SLG_zero = np.asarray(ZT_SLG_zero)

If_pu_SLG_zero = 1.0 / ZT_SLG_zero
Tp_SLG_zero = np.where(ZT_SLG_zero.real != 0, ZT_SLG_zero.imag / ZT_SLG_zero.real, np.nan)


# Base currents
I_base = dfs["Nodes"]["Base Current [pu]"].values  # A

# Stack I0, I1, I2 for each node
I_seq = np.vstack([If_pu_SLG, If_pu_SLG, If_pu_SLG])         # SLG with Zf
I_seq_zero = np.vstack([If_pu_SLG_zero, If_pu_SLG_zero, If_pu_SLG_zero])  # bolted fault

# Phase currents in pu
I_abc_pu = (A @ I_seq) / 3
I_abc_pu_zero = (A @ I_seq_zero) / 3

# Convert to Amperes
I_abc_A = I_abc_pu * I_base
I_abc_A_zero = I_abc_pu_zero * I_base

# Compute RMS and phase angle
I_abc_rms = np.abs(I_abc_A) / np.sqrt(2)
I_abc_rms_zero = np.abs(I_abc_A_zero) / np.sqrt(2)

I_abc_angle = np.angle(I_abc_A, deg=True)
I_abc_angle_zero = np.angle(I_abc_A_zero, deg=True)

# Build DataFrame
n_nodes = I_abc_A.shape[1]
df_SLG = pd.DataFrame({
    "Node": np.repeat(np.arange(n_nodes), 3),
    "SLG Phase": np.tile(["A", "B", "C"], n_nodes),
    "SLG Irms Min[A]": I_abc_rms.T.flatten(),
    "SLG Phase Min[deg]": I_abc_angle.T.flatten(),
    "SLG Tp Min[s]": np.repeat(Tp_SLG, 3),
    "SLG Irms Max[A]": I_abc_rms_zero.T.flatten(),
    "SLG Phase Max[deg]": I_abc_angle_zero.T.flatten(),
    "SLG Tp Max[s]": np.repeat(Tp_SLG_zero, 3)
})

df_SLG.head(9)


,Node,SLG Phase,SLG Irms Min[A],SLG Phase Min[deg],SLG Tp Min[s],SLG Irms Max[A],SLG Phase Max[deg],SLG Tp Max[s]
0,0,A,5.060544e+02,-6.070470,0.106348,4.783716e+03,-88.519828,38.700246
1,0,B,5.631390e-14,107.308253,0.106348,5.862404e-13,14.121845,38.700246
2,0,C,6.757473e-14,97.241046,0.106348,6.403519e-13,20.468068,38.700246
3,1,A,4.981631e+02,-9.758790,0.171989,2.934557e+03,-86.848118,18.159937
4,1,B,6.278654e-14,95.134722,0.171989,3.687035e-13,19.436775,18.159937
5,1,C,6.064935e-14,93.000906,0.171989,3.024642e-13,20.187621,18.159937
6,2,A,4.244490e+02,-27.885030,0.529138,9.019386e+02,-83.638691,8.969878
7,2,B,4.783048e-14,80.644985,0.529138,1.145148e-13,22.738651,8.969878
8,2,C,4.827383e-14,77.494549,0.529138,1.196686e-13,24.889674,8.969878


## Line to Line fault

$$\bar{I}_{1-\Delta k} = - \bar{I}_{2-\Delta k} = \frac{\bar{V}_{1-pF}}{\bar{Z}_{1-kk} + \bar{Z}_{2-kk} + 3R_f} = \frac{\bar{V}_{1-pF}}{\bar{Z}_T}$$

In [85]:
# Fault current with Zf_max_pu
ZT_LL = np.diag(Z1) + np.diag(Z1) + Zf_max_pu
ZT_LL = np.asarray(ZT_LL)  # make sure it's a NumPy array
If_pu_LL = 1.0 / ZT_LL  # pu assuming V_prefault = 1 pu
Tp_LL = np.where(ZT_LL.real != 0, ZT_LL.imag / ZT_LL.real, np.nan)

# Base currents
I_base = dfs["Nodes"]["Base Current [pu]"].values  # A

# Stack I0, I1, I2 for each node
I_seq = np.vstack([np.zeros_like(If_pu_LL), If_pu_LL, -If_pu_LL])

# Phase currents in pu
I_abc_pu = (A @ I_seq) / 3

# Convert to Amperes
I_abc_A = I_abc_pu * I_base


# Compute RMS and phase angle
I_abc_rms = np.abs(I_abc_A) / np.sqrt(2)

I_abc_angle = np.angle(I_abc_A, deg=True)

# Build DataFrame
n_nodes = I_abc_A.shape[1]
df_LL = pd.DataFrame({
    "Node": np.repeat(np.arange(n_nodes), 3),
    "LL Phase": np.tile(["A", "B", "C"], n_nodes),
    "LL Irms[A]": I_abc_rms.T.flatten(),
    "LL Phase[deg]": I_abc_angle.T.flatten(),
    "LLTp[s]": np.repeat(Tp_LL, 3)
})

df_LL.head(9)


,Node,LL Phase,LL Irms[A],LL Phase[deg],LLTp[s]
0,0,A,0.000000,0.000000,0.257956
1,0,B,850.306938,-104.464448,0.257956
2,0,C,850.306938,75.535552,0.257956
3,1,A,0.000000,0.000000,0.398609
4,1,B,803.235912,-111.732668,0.398609
5,1,C,803.235912,68.267332,0.398609
6,2,A,0.000000,0.000000,0.742256
7,2,B,658.780042,-126.584882,0.742256
8,2,C,658.780042,53.415118,0.742256


## Final Table 1:

In [86]:
df_TP = df_TP.iloc[::3]
df_SLG = df_SLG.iloc[::3]
df_LL = df_LL.iloc[1::3]

df_TP["Node"] = df_TP["Node"] + 1
df_SLG["Node"] = df_SLG["Node"] + 1
df_LL["Node"] = df_LL["Node"] + 1

combined_df = df_TP.merge(df_SLG, on="Node").merge(df_LL, on="Node")

combined_df

,Node,TP Phase,TP Irms[A],TP Phase[deg],TP Tp[s],SLG Phase,SLG Irms Min[A],SLG Phase Min[deg],SLG Tp Min[s],SLG Irms Max[A],SLG Phase Max[deg],SLG Tp Max[s],LL Phase,LL Irms[A],LL Phase[deg],LLTp[s]
0,1,A,3929.624936,-88.557393,39.708443,A,506.054374,-6.070470,0.106348,4783.716180,-88.519828,38.700246,B,850.306938,-104.464448,0.257956
1,2,A,2501.171023,-86.882959,18.363329,A,498.163086,-9.758790,0.171989,2934.557281,-86.848118,18.159937,B,803.235912,-111.732668,0.398609
2,3,A,1270.374794,-84.475169,10.338432,A,424.449013,-27.885030,0.529138,901.938562,-83.638691,8.969878,B,658.780042,-126.584882,0.742256
3,4,A,1128.794785,-83.850010,9.280596,A,405.336260,-30.996548,0.600779,781.274623,-83.035916,8.186769,B,623.249454,-129.337245,0.819577
4,5,A,1010.408787,-84.111723,9.696202,A,386.565255,-34.770098,0.694244,673.625457,-83.603062,8.919504,B,592.215261,-132.315758,0.910433
5,6,A,936.554992,-84.814497,11.019038,A,379.408317,-36.675207,0.744704,632.198703,-84.401414,10.201381,B,572.387933,-134.653870,0.987990
6,7,A,1977.937557,-87.511855,23.013035,A,494.131510,-12.004796,0.212644,2373.054825,-87.293014,21.150146,B,771.940133,-116.758198,0.504221
7,8,A,311.610893,-83.581849,8.889777,A,173.976357,-63.623057,2.016524,192.914294,-83.419314,8.668339,B,250.402221,-157.230773,2.382487
8,9,A,231.857238,-84.248634,9.928634,A,133.661161,-69.041363,2.610721,142.384009,-84.144338,9.750589,B,191.699338,-161.786680,3.039136
9,10,A,91167.321620,-89.455551,105.233053,A,34.020608,-0.022167,0.000387,87933.375052,-89.649912,163.659185,B,58.925131,-90.042760,0.000746
